# Intraday Option Vol Surfaces

In [214]:
import os
import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import date

from datetime import timedelta, datetime, date
from polygon import RESTClient
import datetime as date
import plotly.graph_objects as go
from plotly.offline import plot

polygonAPIkey = 'x4iF2cYfBrBHLTUlwweq08IBJaDM0L3t'

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import plotly.offline as py

from scipy.interpolate import griddata
from py_vollib_vectorized import vectorized_implied_volatility as implied_volatility

In [188]:
client = RESTClient(polygonAPIkey)

def get_contracts_overview(underlying,limit=1000):
    """
    Input:
        underlying: (string) , ticker for the underlying stock
        limit: (int) , data per page
    """
    contracts = []
    
    for c in client.list_options_contracts(underlying_ticker=underlying,limit=limit):
        contracts.append(c)
    
    contracts = pd.DataFrame(contracts)
    
    return contracts

def get_specific_contract_history(contract_object,multiplier,timespan,start_date,end_date,to_plot=False):
    """_summary_

    Args:
        contract_object (from get_contracts_overvview): return of the get_contracts_overview
        multiplier (int): frequency based on time span
        timespan (int): 'day','week','hour','minute'
        start_date (string): start date
        end_date (string): end date
        plot (boolean): to plot data or not
    """
    optionsTicker = contract_object
    ticker = optionsTicker.underlying_ticker 
    strike = optionsTicker.strike_price
    num_shares = optionsTicker.shares_per_contract
    oType = optionsTicker.contract_type
    oStyle = optionsTicker.exercise_style
    expiryDate = optionsTicker.expiration_date 
    optionData = client.get_aggs(ticker = ticker,
                                 multiplier = multiplier,
                                 timespan = timespan,
                                 from_ = start_date,
                                 to = end_date)
    
    
    optionDataFrame = pd.DataFrame(optionData)
    optionDataFrame['Date'] = optionDataFrame['timestamp'].apply(lambda x:pd.to_datetime(x*1000000))
    optionDataFrame = optionDataFrame.set_index('Date')
    
    fig = go.Figure(data=[go.Candlestick(x=optionDataFrame.index,
                                         open = optionDataFrame['open'],
                                         high = optionDataFrame['high'],
                                         low = optionDataFrame['low'],
                                         close = optionDataFrame['close'])],
                    layout=go.Layout(
                        title=go.layout.Title(text="Option Price for {} {} on {} with Strike {} for {} shares expirying(ed) on {}".format(oStyle,oType,ticker,strike,num_shares,expiryDate))
                    ))
    
    
    if to_plot:
        py.plot(fig,auto_open=True)
    
    print("Option Price for {} {} on {} with Strike {} for {} shares expirying(ed) on {}".format(oStyle,oType,ticker,strike,num_shares,expiryDate))
    return optionDataFrame

def get_all_expiry_dates(contracts):
    
    return contracts['expiration_date'].unique()
    
def get_all_strikes(contracts):
    
    return contracts['strike_price'].unique()
    
def get_strike_expiry(contracts,expiry_dates):
    """_summary_

    Args:
        contracts (dataframe): result of get_contracts_overview
        expiry_dates (list): result of get_all_expiry_dates
    """
    all_strikes = {}
    for exp_date in expiry_dates:
        all_strikes[exp_date] = pd.to_numeric(get_all_strikes(contracts))
        
    return all_strikes

def get_vol_surface_strikes(get_strike_expiry_object):
    """_summary_
    goal: avoid the situation where strike would not be uniform across expiry sate and get rectangular matrix
    Args:
        get_strike_expiry_object (dict): result of get_strike_expiry

    Returns:
        _type_: _description_
    """
    all_strikes = get_strike_expiry_object
    combined_strikes = [list(strikes) for strikes in all_strikes.values()]
    vol_surface_strikes = set.intersection(*map(set,combined_strikes))
    
    return vol_surface_strikes

In [189]:
df = get_contracts_overview("AAPL",limit=1000);
df

,additional_underlyings,cfi,contract_type,correction,exercise_style,expiration_date,primary_exchange,shares_per_contract,strike_price,ticker,underlying_ticker
0,None,OCASPS,call,None,american,2023-01-20,BATO,100,30.0,O:AAPL230120C00030000,AAPL
1,None,OCASPS,call,None,american,2023-01-20,BATO,100,35.0,O:AAPL230120C00035000,AAPL
2,None,OCASPS,call,None,american,2023-01-20,BATO,100,40.0,O:AAPL230120C00040000,AAPL
3,None,OCASPS,call,None,american,2023-01-20,BATO,100,50.0,O:AAPL230120C00050000,AAPL
4,None,OCASPS,call,None,american,2023-01-20,BATO,100,55.0,O:AAPL230120C00055000,AAPL
...,...,...,...,...,...,...,...,...,...,...,...
2111,None,OPASPS,put,None,american,2025-06-20,BATO,100,230.0,O:AAPL250620P00230000,AAPL
2112,None,OPASPS,put,None,american,2025-06-20,BATO,100,235.0,O:AAPL250620P00235000,AAPL
2113,None,OPASPS,put,None,american,2025-06-20,BATO,100,240.0,O:AAPL250620P00240000,AAPL
2114,None,OPASPS,put,None,american,2025-06-20,BATO,100,245.0,O:AAPL250620P00245000,AAPL


In [221]:
c = get_all_expiry_dates(df)
d = get_strike_expiry(df,c)
e = get_vol_surface_strikes(d)


In [213]:
a = df.loc[1000]
a.expiration_date
a
b = get_specific_contract_history(a,5,'minute','2023-01-13','2023-01-13',to_plot=False)
b

Option Price for american put on AAPL with Strike 195.0 for 100 shares expirying(ed) on 2023-04-21


,open,high,low,close,volume,vwap,timestamp,transactions,otc
Date,,,,,,,,,
2023-01-13 09:00:00,133.43,133.60,133.41,133.42,5852.0,133.4613,1673600400000,153,None
2023-01-13 09:05:00,133.39,133.43,133.35,133.35,4050.0,133.3907,1673600700000,82,None
2023-01-13 09:10:00,133.33,133.40,133.32,133.40,4440.0,133.3817,1673601000000,85,None
2023-01-13 09:15:00,133.35,133.35,133.25,133.30,2473.0,133.3213,1673601300000,80,None
2023-01-13 09:20:00,133.30,133.30,133.25,133.25,2667.0,133.2682,1673601600000,68,None
...,...,...,...,...,...,...,...,...,...
2023-01-14 00:35:00,134.56,134.58,134.55,134.58,4981.0,134.5530,1673656500000,81,None
2023-01-14 00:40:00,134.56,134.58,134.56,134.56,2672.0,134.5723,1673656800000,57,None
2023-01-14 00:45:00,134.56,134.56,134.56,134.56,264.0,134.5636,1673657100000,15,None
